# LoRA Fine-Tuning Demo with SmolLM2-360M

This notebook demonstrates:
1. Testing the base model on sample inputs
2. Fine-tuning using PEFT/LoRA on IMDB sentiment dataset
3. Comparing base vs fine-tuned model performance
4. Saving and loading the fine-tuned model

**Base Model:** HuggingFaceTB/SmolLM2-360M-Instruct  
**Dataset:** shawhin/imdb-truncated (1000 train, 1000 validation samples)

## 1. Setup and Installation

In [1]:
# Install required packages
!pip install -q transformers datasets peft accelerate bitsandbytes trl torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 17.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
import numpy as np
from datetime import datetime
import json

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## 2. Load Dataset and Inspect

In [ ]:
# Load the IMDB dataset
dataset = load_dataset('shawhin/imdb-truncated')
print(dataset)
print("\nSample from training set:")
print(dataset['train'][0])

## 3. Load Base Model and Tokenizer

In [ ]:
# Model configuration
model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"Model loaded: {model_name}")
print(f"Model parameters: {base_model.num_parameters():,}")

## 4. Test Base Model (Before Fine-Tuning)

Let's test the base model on some sentiment analysis tasks to establish a baseline.

In [ ]:
# Test prompts - NOT from training data
test_prompts = [
    "Review: This restaurant exceeded all my expectations. The food was delicious and the service was impeccable.\nSentiment:",
    "Review: Terrible experience. The product broke after one day and customer service was unhelpful.\nSentiment:",
    "Review: Average movie, nothing special but not terrible either. Worth watching if you have time.\nSentiment:",
    "Review: Absolutely loved this book! Couldn't put it down, read it in one sitting.\nSentiment:",
    "Review: Waste of money. Poor quality and doesn't work as advertised.\nSentiment:"
]

def generate_response(model, prompt, max_new_tokens=50):
    """Generate response from model"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

print("=" * 80)
print("BASE MODEL OUTPUTS (Before Fine-Tuning)")
print("=" * 80)

base_outputs = []
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- Test {i} ---")
    print(f"Prompt: {prompt}")
    output = generate_response(base_model, prompt)
    base_outputs.append(output)
    print(f"Output: {output}")
    print("-" * 80)

## 5. Prepare Dataset for Fine-Tuning

Format the IMDB dataset for sentiment analysis training.

In [ ]:
def create_prompt(example):
    """Create instruction-formatted prompt for sentiment analysis"""
    sentiment = "positive" if example['label'] == 1 else "negative"

    # Instruction format
    prompt = f"Review: {example['text']}\nSentiment: {sentiment}"

    return {"text": prompt}

# Format datasets
formatted_train = dataset['train'].map(create_prompt, remove_columns=['label'])
formatted_val = dataset['validation'].map(create_prompt, remove_columns=['label'])

print("Sample formatted training example:")
print(formatted_train[0]['text'][:200] + "...")

In [ ]:
def tokenize_function(examples):
    """Tokenize the text data"""
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        max_length=256,
        padding="max_length",
        return_tensors=None
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize datasets
tokenized_train = formatted_train.map(
    tokenize_function,
    batched=True,
    remove_columns=['text']
)

tokenized_val = formatted_val.map(
    tokenize_function,
    batched=True,
    remove_columns=['text']
)

print(f"\nTokenized training samples: {len(tokenized_train)}")
print(f"Tokenized validation samples: {len(tokenized_val)}")

## 6. Configure LoRA and PEFT

Set up Low-Rank Adaptation (LoRA) configuration for efficient fine-tuning.

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=16,                          # Rank of the low-rank matrices
    lora_alpha=32,                 # Scaling factor
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Modules to apply LoRA
    lora_dropout=0.05,             # Dropout probability
    bias="none",                   # Bias training strategy
    task_type=TaskType.CAUSAL_LM   # Task type
)

print("LoRA Configuration:")
print(lora_config)

In [ ]:
# Create a fresh model for training
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

## 7. Configure Training Arguments and Trainer

In [ ]:
# Output directory for checkpoints
output_dir = "./lora_finetuned_smollm2"

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_steps=100,
    logging_steps=50,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=True,
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

print("Trainer initialized successfully!")

## 8. Train the Model

This will take several minutes depending on your hardware.

In [ ]:
print("Starting training...")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Train the model
trainer.train()

print(f"\nTraining completed!")
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 9. Save the Fine-Tuned Model

In [ ]:
# Save the LoRA adapter
lora_adapter_path = "./lora_adapter_smollm2_sentiment"
model.save_pretrained(lora_adapter_path)
tokenizer.save_pretrained(lora_adapter_path)

print(f"LoRA adapter saved to: {lora_adapter_path}")

# Save training metadata
metadata = {
    "base_model": model_name,
    "dataset": "shawhin/imdb-truncated",
    "train_samples": len(tokenized_train),
    "val_samples": len(tokenized_val),
    "lora_config": {
        "r": lora_config.r,
        "lora_alpha": lora_config.lora_alpha,
        "target_modules": lora_config.target_modules,
        "lora_dropout": lora_config.lora_dropout
    },
    "training_args": {
        "num_epochs": training_args.num_train_epochs,
        "learning_rate": training_args.learning_rate,
        "batch_size": training_args.per_device_train_batch_size
    },
    "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open(f"{lora_adapter_path}/training_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("Training metadata saved!")

## 10. Test Fine-Tuned Model and Compare

Load the fine-tuned model and compare its performance with the base model.

In [ ]:
from peft import PeftModel

# Load base model again (fresh)
base_model_test = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Load fine-tuned model (base + LoRA adapter)
finetuned_model = PeftModel.from_pretrained(
    base_model_test,
    lora_adapter_path
)

print("Fine-tuned model loaded successfully!")

In [ ]:
print("=" * 80)
print("MODEL COMPARISON: Base vs Fine-Tuned")
print("=" * 80)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n{'='*80}")
    print(f"Test {i}")
    print(f"{'='*80}")
    print(f"Prompt: {prompt}\n")

    # Base model output
    print("BASE MODEL OUTPUT:")
    base_output = generate_response(base_model_test, prompt)
    print(base_output)

    print("\n" + "-" * 80 + "\n")

    # Fine-tuned model output
    print("FINE-TUNED MODEL OUTPUT:")
    finetuned_output = generate_response(finetuned_model, prompt)
    print(finetuned_output)

    print("\n" + "=" * 80)

## 11. Quantitative Evaluation (Optional)

Evaluate both models on the validation set to get numerical metrics.

In [ ]:
def evaluate_sentiment_accuracy(model, test_dataset, num_samples=100):
    """Evaluate model accuracy on sentiment classification"""
    correct = 0
    total = 0

    # Take a subset of validation data that wasn't in training
    samples = test_dataset.select(range(min(num_samples, len(test_dataset))))

    for sample in samples:
        # Create prompt without answer
        review_text = dataset['validation'][total]['text']
        true_label = dataset['validation'][total]['label']
        true_sentiment = "positive" if true_label == 1 else "negative"

        prompt = f"Review: {review_text}\nSentiment:"

        # Generate prediction
        output = generate_response(model, prompt, max_new_tokens=10)

        # Extract predicted sentiment
        output_lower = output.lower()
        if "positive" in output_lower and "negative" not in output_lower:
            predicted_sentiment = "positive"
        elif "negative" in output_lower and "positive" not in output_lower:
            predicted_sentiment = "negative"
        else:
            predicted_sentiment = None

        if predicted_sentiment == true_sentiment:
            correct += 1

        total += 1

        if total % 20 == 0:
            print(f"Processed {total}/{num_samples} samples...")

    accuracy = correct / total if total > 0 else 0
    return accuracy, correct, total

print("Evaluating Base Model...")
base_acc, base_correct, base_total = evaluate_sentiment_accuracy(base_model_test, tokenized_val, num_samples=50)

print("\nEvaluating Fine-Tuned Model...")
ft_acc, ft_correct, ft_total = evaluate_sentiment_accuracy(finetuned_model, tokenized_val, num_samples=50)

print("\n" + "=" * 80)
print("EVALUATION RESULTS")
print("=" * 80)
print(f"Base Model Accuracy: {base_acc:.2%} ({base_correct}/{base_total})")
print(f"Fine-Tuned Model Accuracy: {ft_acc:.2%} ({ft_correct}/{ft_total})")
print(f"Improvement: {(ft_acc - base_acc):.2%}")
print("=" * 80)

## 12. Summary and Next Steps

### What we accomplished:
1. ✅ Tested the base SmolLM2-360M model on sentiment analysis
2. ✅ Fine-tuned using LoRA/PEFT on 1000 IMDB reviews
3. ✅ Compared base vs fine-tuned model performance
4. ✅ Saved the LoRA adapter for future use

### Key Takeaways:
- LoRA allows efficient fine-tuning with minimal trainable parameters
- The fine-tuned model should show improved sentiment classification
- Test prompts were kept separate from training data to ensure fair evaluation

### To use the fine-tuned model later:
```python
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")
model = PeftModel.from_pretrained(base_model, "./lora_adapter_smollm2_sentiment")
tokenizer = AutoTokenizer.from_pretrained("./lora_adapter_smollm2_sentiment")
```